<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/FastText_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fasttext

In [ ]:
import fasttext.util
from gensim.models.wrappers import FastText

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec

Load pretrained model: wiki sample

In [ ]:
model = KeyedVectors.load_word2vec_format('wiki.simple.vec')

Try to see the vector of few words:

In [77]:
# model['word'] --> access keyed vectors
model['house']

array([ 2.0695e-01,  7.9656e-02,  4.3752e-01,  1.1519e-01, -4.2894e-02,
        5.0550e-01, -1.9424e-02,  2.2890e-01,  2.2018e-01, -1.2672e-01,
       -1.1345e-01,  3.3551e-01,  4.6808e-01,  3.7312e-01, -2.5111e-01,
       -1.1842e-01,  1.6301e-01,  4.3348e-02, -4.7680e-01,  1.8059e-01,
        1.5040e-01,  1.5900e-01, -1.0340e-01, -5.9205e-02, -2.1054e-01,
       -1.8093e-01,  1.7380e-01,  3.3910e-02, -2.0083e-01,  3.3837e-01,
        4.6986e-01,  1.1231e-01, -3.9199e-03, -3.4872e-02,  2.2115e-01,
       -2.8744e-01, -3.9006e-01, -2.4090e-01, -1.6768e-01,  4.4849e-01,
        1.3874e-01, -4.9120e-02,  3.4582e-01,  1.5075e-01,  3.7149e-01,
        3.7103e-01, -1.4188e-01, -1.3651e-01,  8.6161e-02, -5.4532e-01,
        6.1135e-01, -7.0246e-03,  6.2059e-02,  2.5860e-02,  1.1958e-02,
       -1.2276e-01,  4.2305e-01, -1.3935e-01,  2.2550e-01,  2.2193e-01,
       -1.5392e-01,  3.7199e-01, -2.1911e-01,  6.4260e-04, -9.6433e-02,
       -3.8005e-01, -4.6484e-02, -1.9214e-01,  9.3353e-02,  1.16

In [76]:
model['HOUSE']

KeyError: ignored

Handle exception: word not in vocabulary

In [ ]:
try:
  model.wv['HOUSE']
except:
  print ("word not in vocabulary")

word not in vocabulary


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Try to calculate cosine similarity:

In [ ]:
from scipy import spatial

In [45]:
word1 = model.wv['house']
word2 = model.wv['home']

cosine_similarity = 1 - spatial.distance.cosine(word1, word2)
print(cosine_similarity)

0.3566957712173462


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Let's work with the dataframe

In [46]:
import numpy as np
import pandas as pd

Load the dataframe from csv

In [83]:
df = pd.read_csv('df_combinations.csv')

In [84]:
df.head()

,Concepticon_Gloss_1,Concepticon_Gloss_2,Concepticon_ID_1,Concepticon_ID_2
0,gold,good,1369,1035
1,gold,green,1369,1425
2,gold,hammer,1369,1481
3,gold,hand,1369,1277
4,gold,head,1369,1256


In [78]:
# inserted error to handle not in vocabulary exception
df['Concepticon_Gloss_1'].loc[0]='gooold'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Let's add the column with cosine similarity

In [81]:
# add new column with default 0
df['cosine_similarity']=0

# for every word in column1 and column2, calculate cosine similarity
for i in range(len(df['Concepticon_Gloss_1'])):
  try:
    w1 = df['Concepticon_Gloss_1'].loc[i]
    vec1 = model[w1]
    w2 = df['Concepticon_Gloss_2'].loc[i]
    vec2 = model[w2]
    cosine_similarity = 1 - spatial.distance.cosine(vec1, vec2)
  except:
    # if the word is not in vocab, cosine similarity is 0
    cosine_similarity=0
  df['cosine_similarity'].loc[i]=cosine_similarity


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [82]:
df.head()

,Concepticon_Gloss_1,Concepticon_Gloss_2,Concepticon_ID_1,Concepticon_ID_2,cosine_similarity
0,gooold,good,1369,1035,0.000000
1,gold,green,1369,1425,0.185446
2,gold,hammer,1369,1481,0.228490
3,gold,hand,1369,1277,0.112822
4,gold,head,1369,1256,0.105320
